## Lab 2: 고객 선호도 메모리로 에이전트 개인화

### 개요

Lab 1에서는 로컬 세션에서 단일 사용자에게 잘 작동하는 패션/뷰티 고객 지원 에이전트를 구축했습니다. 하지만 실제 패션/뷰티 이커머스에서는 단일 사용자가 로컬 환경에서 실행하는 것 이상의 기능이 필요합니다.

**프로덕션 환경에서 에이전트를 실행**할 때 필요한 사항들:
- **다중 사용자 지원**: 수천 명의 고객을 동시에 처리
- **영구 저장소**: 세션 생명주기를 넘어선 대화 저장
- **장기 학습**: 고객 선호도 및 행동 패턴 추출
- **세션 간 연속성**: 다른 상호작용에서 고객 기억

**워크숍 진행 상황:**
- **Lab 1 (완료)**: 에이전트 프로토타입 - 기능적인 패션/뷰티 고객 지원 에이전트 구축
- **Lab 2 (현재)**: 메모리로 강화 - 대화 맥락 및 개인화 추가
- **Lab 3**: Gateway & Identity로 확장 - 에이전트 간 도구 안전하게 공유
- **Lab 4**: 프로덕션 배포 - AgentCore Runtime으로 관측성 확보
- **Lab 5**: 사용자 인터페이스 구축 - 고객 대상 애플리케이션 생성

이 랩에서는 골드피시 에이전트(몇 초 만에 대화를 잊음)를 스마트한 개인화 어시스턴트로 변환하는 누락된 지속성 및 학습 레이어를 추가하겠습니다.

메모리는 지능의 핵심 구성요소입니다. 대형 언어 모델(LLM)은 인상적인 능력을 가지고 있지만 대화 전반에 걸친 영구적인 메모리가 부족합니다. [Amazon Bedrock AgentCore Memory](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory-getting-started.html)는 AI 에이전트가 시간이 지남에 따라 맥락을 유지하고, 중요한 사실을 기억하며, 일관되고 개인화된 경험을 제공할 수 있도록 하는 관리형 서비스를 제공하여 이러한 한계를 해결합니다.

AgentCore Memory는 두 수준에서 작동합니다:
- **단기 메모리**: 즉시 대화 맥락 및 세션 기반 정보로 단일 상호작용 또는 밀접하게 관련된 세션 내에서 연속성을 제공합니다.
- **장기 메모리**: 여러 대화에서 추출되고 저장된 영구적인 정보로, 시간이 지남에 따라 개인화된 경험을 가능하게 하는 사실, 선호도 및 요약을 포함합니다.

### Lab 2를 위한 아키텍처
<div style="text-align:left">
    <img src="images/architecture_lab2_memory.png" width="75%"/>
</div>

*영구적인 단기 및 장기 메모리 기능을 갖춘 다중 사용자 에이전트*


### Step 1: 라이브러리 가져오기

AgentCore Memory를 위한 라이브러리를 가져오겠습니다. 이를 위해 AgentCore 기능 작업을 돕는 경량 래퍼인 [Amazon Bedrock AgentCore Python SDK](https://github.com/aws/bedrock-agentcore-sdk-python)를 사용하겠습니다.

In [ ]:
import logging
import sys
import os

# 프로젝트 루트 경로를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# AgentCore Memory 가져오기
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent

import boto3
from boto3.session import Session

boto_session = Session()
REGION = boto_session.region_name

logger = logging.getLogger(__name__)

# 이제 lab_helpers를 import할 수 있습니다
import sys; sys.path.insert(0, '..'); from src.helpers.utils import get_ssm_parameter, put_ssm_parameter

print(f"✅ 라이브러리 가져오기 완료. 사용 리전: {REGION}")
print(f"📂 프로젝트 루트: {project_root}")

### Step 2: Bedrock AgentCore Memory 리소스 생성

Amazon Bedrock AgentCore Memory는 여러 장기 메모리 전략을 제공합니다. 다음을 결합한 메모리 리소스를 생성합니다:

- **USER_PREFERENCE**: 고객 선호도 및 행동 추출
- **SEMANTIC**: 벡터 임베딩을 사용하여 사실 정보 저장

AgentCore Memory는 네임스페이스를 사용하여 장기 메모리 메시지를 논리적으로 그룹화합니다. 이 메모리 전략을 사용하여 새로운 장기 메모리가 추출될 때마다 설정한 네임스페이스 아래에 저장됩니다. `actorId`를 사용하여 동일한 고객의 메시지를 함께 그룹화하는 다음 네임스페이스를 사용합니다:

- `ecommerce/customer/{actorId}/preferences`: 사용자 선호도 메모리 전략용
- `ecommerce/customer/{actorId}/history`: 시맨틱 메모리 전략용

In [ ]:
memory_client = MemoryClient(region_name=REGION)
memory_name = "EcommerceCustomerMemory"

def create_or_get_ecommerce_memory_resource():
    try:
        # 기존 메모리 ID 확인
        memory_id = get_ssm_parameter("/app/ecommerce/agentcore/memory_id")
        memory_client.gmcp_client.get_memory(memoryId=memory_id)
        return memory_id
    except:
        try:
            # 이커머스 특화 메모리 전략
            strategies = [
                {
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "EcommerceCustomerPreferences",
                        "description": "고객의 패션/뷰티 선호도, 사이즈, 브랜드, 스타일 등을 저장",
                        "namespaces": ["ecommerce/customer/{actorId}/preferences"],
                    }
                },
                {
                    StrategyType.SEMANTIC.value: {
                        "name": "EcommerceCustomerHistory", 
                        "description": "고객의 구매 이력, 반품/교환 내역, 문의 사항 저장",
                        "namespaces": ["ecommerce/customer/{actorId}/history"],
                    }
                },
            ]
            
            print("이커머스 AgentCore Memory 리소스 생성 중... 몇 분 소요될 수 있습니다.")
            
            # *** AGENTCORE MEMORY 사용 *** - 시맨틱 전략으로 메모리 리소스 생성
            response = memory_client.create_memory_and_wait(
                name=memory_name,
                description="패션/뷰티 이커머스 고객 지원 메모리",
                strategies=strategies,
                event_expiry_days=90,  # 메모리는 90일 후 만료
            )
            
            memory_id = response["id"]
            try:
                put_ssm_parameter("/app/ecommerce/agentcore/memory_id", memory_id)
            except:
                raise
            return memory_id
        except:
            return None

In [ ]:
memory_id = create_or_get_ecommerce_memory_resource()
print("✅ 이커머스 AgentCore Memory 생성 완료")

생성된 메모리: 
![agentcore_memory_console.png](images/agentcore_memory_console.png)

## Step 3: 이전 고객 데이터 메모리 저장

`create_event` 액션은 에이전트 상호작용을 단기 메모리에 즉시 저장합니다. 저장된 각 상호작용에는 사용자 메시지, 어시스턴트 응답 및 도구 액션이 포함될 수 있습니다. 이 프로세스는 동기식으로 진행되어 대화 데이터가 손실되지 않도록 보장합니다.

그런 다음 단기 메모리 메시지는 선택된 장기 메모리 전략에 따라 비동기적으로 처리됩니다.

`actor_id`로 고객 ID를 제공하고 `session_id`를 제공하여 이전 고객 상호작용을 로드해보겠습니다.

In [ ]:
# 기존 메모리 리소스 나열
for memory in memory_client.list_memories():
    print(f"메모리 ARN: {memory.get('arn')}")
    print(f"메모리 ID: {memory.get('id')}")
    print("--------------------------------------------------------------------")

# 이전 고객 상호작용으로 시드
CUSTOMER_ID = "customer_ecommerce_001"

# 패션/뷰티 특화 이전 상호작용
previous_interactions = [
    ("지난달에 산 원피스 사이즈가 작아서 L로 교환했어요.", "USER"),
    ("사이즈 교환 처리해드렸습니다. 고객님께는 보통 L 사이즈가 잘 맞으시는 것 같아요. 플라워 패턴이 정말 잘 어울리실 것 같습니다!", "ASSISTANT"),
    
    ("제가 건성 피부인데 어떤 파운데이션이 좋을까요?", "USER"), 
    ("건성 피부에는 보습 쿠션이나 글로우 타입을 추천드립니다. 히알루론산이나 세라마이드 성분이 들어간 제품이 특히 좋아요.", "ASSISTANT"),
    
    ("평소에 M 사이즈 입는데 이 브랜드는 어떤가요?", "USER"),
    ("해당 브랜드는 사이즈가 작게 나오는 편이니 L 사이즈를 추천드립니다. 상품 상세페이지의 실측 사이즈를 꼭 확인해보세요!", "ASSISTANT"),
    
    ("베이지색을 좋아하는데 어떤 옷과 잘 어울릴까요?", "USER"),
    ("베이지는 정말 활용도가 높은 색상이에요! 화이트, 네이비, 블랙 등 어떤 색과도 잘 어울리고, 특히 가을 시즌에 완벽합니다.", "ASSISTANT"),
    
    ("이 립스틱 색깔이 사진과 너무 달라요. 교환 가능한가요?", "USER"),
    ("색상 차이로 인한 교환은 무료로 처리됩니다. 어떤 톤을 원하시는지 말씀해주시면 비슷한 색상으로 추천해드릴게요!", "ASSISTANT"),
]

# 이전 상호작용 저장
try:
    memory_client.create_event(
        memory_id=memory_id,
        actor_id=CUSTOMER_ID,
        session_id="previous_ecommerce_session",
        messages=previous_interactions
    )
    print("✅ 이커머스 고객 이력 등록 완료")
except Exception as e:
    print(f"⚠️ 이력 시드 오류: {e}")

### 단기 메모리 조회

In [ ]:
# 단기 메모리에서 대화 이력 조회 (list_events 사용)
events = memory_client.list_events(
    memory_id=memory_id,
    actor_id=CUSTOMER_ID,
    session_id="previous_ecommerce_session",
    max_results=10
)

print("💬 단기 메모리 - 저장된 대화 이력:")
print("=" * 50)
for event in events:
    event_id = event.get('eventId', 'N/A')
    timestamp = event.get('eventTimestamp', 'N/A')
    print(f"이벤트 ID: {event_id}")
    print(f"타임스탬프: {timestamp}")
    
    # 메시지 내용 출력
    for payload_item in event.get('payload', []):
        if 'conversational' in payload_item:
            conv = payload_item['conversational']
            role = conv.get('role', 'N/A')
            text = conv.get('content', {}).get('text', '')[:50]
            print(f"  [{role}] {text}...")
    print("-" * 30)

`create_event`를 통해 이벤트를 생성하면 메시지가 단기 메모리로 전송되고 추가로 [장기 메모리](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/long-term-memory.html)로 비동기적으로 전송됩니다.
장기 메모리로 정보가 전파되는 데 약 30초가 걸립니다.

### 선호도 메모리 시각화

In [ ]:
import time
time.sleep(20)  # 메모리 전파를 위한 시간 확보

In [ ]:
# 고객 선호도 메모리 조회
memories = memory_client.retrieve_memories(
    memory_id=memory_id,
    namespace=f"ecommerce/customer/{CUSTOMER_ID}/preferences",
    query="고객의 패션 선호도와 사이즈 정보를 알려주세요"
)

print("🛍️ 고객 선호도 정보:")
print("=" * 50)
for i, memory in enumerate(memories, 1):
    if isinstance(memory, dict):
        content = memory.get('content', {})
        if isinstance(content, dict):
            text = content.get('text', '')
            print(f"  {i}. {text}")
            print()

메모리 Observability 화면:
![memory_observability.png](images/memory_observability.png)

### 구매/문의 이력 메모리 확인

In [ ]:
# 고객 구매/문의 이력 조회
memories = memory_client.retrieve_memories(
    memory_id=memory_id,
    namespace=f"ecommerce/customer/{CUSTOMER_ID}/history",
    query="고객의 구매 이력과 문의 내역을 주세요"
)    

print("📋 고객 구매/문의 이력:")
print("=" * 50)
for i, memory in enumerate(memories, 1):
    if isinstance(memory, dict):
        content = memory.get('content', {})
        if isinstance(content, dict):
            text = content.get('text', '')
            print(f"  {i}. {text}")
            print()

## Step 3: Strands 훅을 구현하여 에이전트 상호작용 저장 및 검색

Strands Agents는 강력한 훅 시스템을 제공하여 구성요소가 강력하게 타입이 지정된 이벤트 콜백을 통해 에이전트 동작에 반응하거나 수정할 수 있습니다. 두 가지 주요 훅 이벤트를 사용하겠습니다:

- **MessageAddedEvent**: 메시지가 대화에 추가될 때 트리거되어 고객 맥락을 검색하고 주입할 수 있습니다
- **AfterInvocationEvent**: 에이전트 응답 후 실행되어 상호작용을 메모리에 자동 저장할 수 있습니다

훅 시스템은 메모리 작업이 수동 개입 없이 자동으로 수행되도록 보장하여 고객 맥락이 대화 전반에 걸쳐 보존되는 원활한 경험을 만듭니다.

`HookProvider` 클래스를 확장하여 훅을 생성하겠습니다:

In [ ]:
class EcommerceCustomerMemoryHooks(HookProvider):
    """이커머스 고객 지원을 위한 메모리 훅"""

    def __init__(
        self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str
    ):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespaces = {
            i["type"]: i["namespaces"][0]
            for i in self.client.get_memory_strategies(self.memory_id)
        }

    def retrieve_customer_context(self, event: MessageAddedEvent):
        """고객 지원 쿼리 처리 전에 고객 맥락 검색"""
        messages = event.agent.messages
        if (
            messages[-1]["role"] == "user"
            and "toolResult" not in messages[-1]["content"][0]
        ):
            user_query = messages[-1]["content"][0]["text"]

            try:
                all_context = []

                for context_type, namespace in self.namespaces.items():
                    # *** AGENTCORE MEMORY 사용 *** - 각 네임스페이스에서 고객 맥락 검색
                    memories = self.client.retrieve_memories(
                        memory_id=self.memory_id,
                        namespace=namespace.format(actorId=self.actor_id),
                        query=user_query,
                        top_k=3,
                    )
                    # 후처리: 메모리를 맥락 문자열로 포맷
                    for memory in memories:
                        if isinstance(memory, dict):
                            content = memory.get("content", {})
                            if isinstance(content, dict):
                                text = content.get("text", "").strip()
                                if text:
                                    # 이커머스 특화 컨텍스트 태그
                                    context_tag = self._get_korean_context_tag(context_type, text)
                                    all_context.append(f"[{context_tag}] {text}")

                # 고객 맥락을 쿼리에 주입
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0][
                        "text"
                    ] = f"고객 정보:\n{context_text}\n\n고객 문의: {original_text}"
                    logger.info(f"고객 맥락 {len(all_context)}개 항목 검색 완료")

            except Exception as e:
                logger.error(f"고객 맥락 검색 실패: {e}")

    def save_ecommerce_interaction(self, event: AfterInvocationEvent):
        """에이전트 응답 후 이커머스 상호작용 저장"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # 마지막 고객 쿼리와 에이전트 응답 가져오기
                customer_query = None
                agent_response = None

                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif (
                        msg["role"] == "user"
                        and not customer_query
                        and "toolResult" not in msg["content"][0]
                    ):
                        customer_query = msg["content"][0]["text"]
                        break

                if customer_query and agent_response:
                    # *** AGENTCORE MEMORY 사용 *** - 이커머스 상호작용 저장
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[
                            (customer_query, "USER"),
                            (agent_response, "ASSISTANT"),
                        ],
                    )
                    logger.info("이커머스 상호작용을 메모리에 저장했습니다")

        except Exception as e:
            logger.error(f"이커머스 상호작용 저장 실패: {e}")

    def register_hooks(self, registry: HookRegistry) -> None:
        """이커머스 고객 지원 메모리 훅 등록"""
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)
        registry.add_callback(AfterInvocationEvent, self.save_ecommerce_interaction)
        logger.info("이커머스 고객 지원 메모리 훅이 등록되었습니다")
    
    def _get_korean_context_tag(self, context_type: str, text: str) -> str:
        """맥락 유형에 따른 한국어 태그를 반환합니다."""
        if context_type.upper() == "USER_PREFERENCE":
            if "사이즈" in text:
                return "선호 사이즈"
            elif "브랜드" in text:
                return "선호 브랜드"
            elif "색상" in text:
                return "선호 색상"
            elif "스타일" in text:
                return "선호 스타일"
            elif "반품" in text or "교환" in text:
                return "반품/교환 이력"
            else:
                return "고객 선호도"
        elif context_type.upper() == "SEMANTIC":
            if "반품" in text:
                return "반품 이력"
            elif "교환" in text:
                return "교환 이력"
            elif "문의" in text:
                return "이전 문의"
            elif "주문" in text:
                return "주문 이력"
            else:
                return "구매 정보"
        else:
            return context_type.upper()

print("✅ 이커머스 메모리 훅 클래스 생성 완료")

## Step 4: 메모리를 갖춘 이커머스 고객 지원 에이전트 생성

다음으로, Lab 1에서와 같이 이커머스 고객 지원 에이전트를 구현하지만, 이번에는 `EcommerceCustomerMemoryHooks` 클래스를 인스턴스화하고 메모리 훅을 에이전트 생성자에 전달합니다.

In [ ]:
import uuid
import sys

from strands import Agent
from strands.models import BedrockModel

sys.path.append('../src')

from agent import (
    SYSTEM_PROMPT,
    process_return, 
    process_exchange,
    web_search,
    MODEL_ID
)

SESSION_ID = str(uuid.uuid4())
memory_hooks = EcommerceCustomerMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

model = BedrockModel(
    model_id=MODEL_ID,
    region_name=REGION
)

agent = Agent(
    model=model,
    hooks=[memory_hooks],
    tools=[
        process_return,
        process_exchange,
        web_search
    ],
    system_prompt=SYSTEM_PROMPT
)

print("✅ 메모리 기능을 갖춘 이커머스 고객 지원 에이전트 생성 완료!")

## Step 7: 메모리 훅 테스트

이제 정교한 메모리 훅 시스템이 어떻게 자동으로 작동하는지 테스트해보겠습니다!

In [ ]:
# 고객 선호도를 반영한 상품 추천 테스트
response1 = agent("안녕하세요! 새 원피스를 찾고 있는데 추천해주세요.")
print("🛍️ 상품 추천 응답:")
print("=" * 50)
print(response1)

In [ ]:
# 고객의 피부 타입을 기억하는지 테스트
response2 = agent("새로운 파운데이션을 사려고 하는데 어떤 게 좋을까요?")
print("💄 뷰티 추천 응답:")
print("=" * 50)
print(response2)

In [ ]:
# 고객의 선호 색상과 스타일을 기억하는지 테스트
response3 = agent("가을에 어울리는 가디건을 추천해주세요. 코디도 알려주세요!")
print("👗 스타일링 조언 응답:")
print("=" * 50)
print(response3)

In [ ]:
# 고객의 사이즈 이력을 기억하는지 테스트
response4 = agent("새 청바지를 주문하려는데 사이즈 조언 부탁드려요.")
print("📐 사이즈 조언 응답:")
print("=" * 50)
print(response4)

## 축하합니다! 🎉

**Lab 2: 이커머스 고객 지원 에이전트에 메모리 추가**를 성공적으로 완료했습니다!

### 달성한 것:

- Amazon Bedrock AgentCore Memory로 서버리스 관리형 메모리 생성
- 고객 선호도 및 시맨틱(사실) 정보를 저장하는 장기 메모리 구현
- Strands Agents에서 제공하는 훅 메커니즘을 사용하여 AgentCore Memory를 이커머스 고객 지원 에이전트와 통합

### 이커머스 특화 성과:

- **고객 선호도 추적**: 선호 사이즈, 색상, 브랜드, 스타일 기억
- **구매 이력 관리**: 이전 주문, 반품/교환 내역 추적
- **개인화된 응답**: 고객별 맞춤 상품 추천 및 사이즈 조언
- **피부 타입 기억**: 뷰티 제품 추천 시 고객의 피부 타입 고려
- **한국어 맥락**: 존댓말과 한국 쇼핑 문화에 맞는 응대

### 현재 시스템의 능력:

- **다중 턴 대화**: 에이전트가 상호작용 전반에 걸쳐 맥락 유지
- **도구 통합**: 제품 정보, 반품 정책, 웹 검색의 원활한 사용
- **메모리 지속성**: 고객 선호도 및 이력 유지
- **실시간 성능**: 성능 메트릭과 함께 스트리밍 응답
- **패션/뷰티 전문성**: 도메인 특화 조언 및 추천

##### 다음 단계 [Lab 3: Gateway로 도구 공유 및 보안 강화 →](lab-03-agentcore-gateway.ipynb)

## 리소스
- [Amazon Bedrock Agent Core Memory](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory.html)
- [Strands Agents 훅 문서](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/hooks/?h=hooks)